# Projeto 4 - Pegando todos os setores da bolsa no site da B3

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from datetime import date
import zipfile
import os

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
url = 'https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm'

In [4]:
driver.get(url)

In [5]:
local_botao_expandir = '/html/body/app-root/app-companies-home/div/div/div/div/div[2]/div[1]/div/div/a/i'


In [6]:
botao_expandir = driver.find_element('xpath', local_botao_expandir)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/app-root/app-companies-home/div/div/div/div/div[2]/div[1]/div/div/a/i"}
  (Session info: chrome=120.0.6099.227); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00DF6EE3+174339]
	(No symbol) [0x00D20A51]
	(No symbol) [0x00A36FF6]
	(No symbol) [0x00A69876]
	(No symbol) [0x00A69C2C]
	(No symbol) [0x00A9BD42]
	(No symbol) [0x00A87054]
	(No symbol) [0x00A9A104]
	(No symbol) [0x00A86DA6]
	(No symbol) [0x00A61034]
	(No symbol) [0x00A61F8D]
	GetHandleVerifier [0x00E94B1C+820540]
	sqlite3_dbdata_init [0x00F553EE+653550]
	sqlite3_dbdata_init [0x00F54E09+652041]
	sqlite3_dbdata_init [0x00F497CC+605388]
	sqlite3_dbdata_init [0x00F55D9B+656027]
	(No symbol) [0x00D2FE6C]
	(No symbol) [0x00D283B8]
	(No symbol) [0x00D284DD]
	(No symbol) [0x00D15818]
	BaseTh

### O site da B3 é só uma casca da antiga BVMF. 

Nesses casos, precisamos trocar pro iframe que importa. 

In [7]:
WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "bvmf_iframe")))

True

In [8]:
botao_expandir = driver.find_element('xpath', local_botao_expandir)

In [9]:
#botao_expandir.click()
driver.execute_script("arguments[0].click();", botao_expandir)

In [10]:
planilha = driver.find_element('xpath', '/html/body/app-root/app-companies-home/div/div/div/div/div[2]/div[2]/div/app-companies-home-filter-classification/form/div[2]/div[3]/div[2]/p/a')

driver.execute_script("arguments[0].click();", planilha)

#time.sleep(5) #tem que esperar baixar a planilha 

In [11]:
driver.quit()

### Agora vamos extrair o ZIP

In [20]:
caminho_downloads = r'C:\Users\ronal\Downloads'

In [21]:
arquivo_zip = zipfile.ZipFile(caminho_downloads + r"\ClassifSetorial.zip")

for planilha in arquivo_zip.namelist():

    setores = pd.read_excel(arquivo_zip.open(planilha), skiprows=6)

In [22]:
arquivo_zip.close()

In [15]:
os.remove(caminho_downloads + r"\ClassifSetorial.zip")

In [23]:
setores

,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM,Unnamed: 4
0,NaN,NaN,NaN,CÓDIGO,SEGMENTO
1,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN,NaN
2,NaN,NaN,3R PETROLEUM,RRRP,NM
3,NaN,NaN,COSAN,CSAN,NM
4,NaN,NaN,ENAUTA PART,ENAT,NM
...,...,...,...,...,...
569,As informações recebidas das empresas admitida...,NaN,NaN,NaN,NaN
570,nosso site www.b3.com.br.,NaN,NaN,NaN,NaN
571,"Para mais esclarecimentos, sugerimos procurar ...",NaN,NaN,NaN,NaN
572,potenciais das negociações com valores mobiliá...,NaN,NaN,NaN,NaN


In [25]:
setores['SUBSETOR'] = setores['SUBSETOR'].ffill()
setores = setores.dropna(subset = ['SEGMENTO'])
setores = setores[['SUBSETOR', 'SEGMENTO', 'LISTAGEM']]
setores['SEGMENTO'] = setores['SEGMENTO'].str.strip()

setores

,SUBSETOR,SEGMENTO,LISTAGEM
1,"Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN
2,"Petróleo, Gás e Biocombustíveis",3R PETROLEUM,RRRP
3,"Petróleo, Gás e Biocombustíveis",COSAN,CSAN
4,"Petróleo, Gás e Biocombustíveis",ENAUTA PART,ENAT
5,"Petróleo, Gás e Biocombustíveis",PET MANGUINH,RPMG
...,...,...,...
551,Outros,POLPAR,PPAR
552,Outros,PROMPT PART,PRPT
553,Outros,SUDESTE S/A,OPSE
554,Outros,SUL 116 PART,OPTS


In [26]:
setores = setores.dropna()
setores.columns = ['SETOR', 'NOME', 'TICKER']
setores

,SETOR,NOME,TICKER
2,"Petróleo, Gás e Biocombustíveis",3R PETROLEUM,RRRP
3,"Petróleo, Gás e Biocombustíveis",COSAN,CSAN
4,"Petróleo, Gás e Biocombustíveis",ENAUTA PART,ENAT
5,"Petróleo, Gás e Biocombustíveis",PET MANGUINH,RPMG
6,"Petróleo, Gás e Biocombustíveis",PETROBRAS,PETR
...,...,...,...
551,Outros,POLPAR,PPAR
552,Outros,PROMPT PART,PRPT
553,Outros,SUDESTE S/A,OPSE
554,Outros,SUL 116 PART,OPTS


In [27]:
setores.to_csv("setores.csv", index = False)